# README.md

Research Question: For the 399 pts with the 899 rows of info, what are the last PSA values before dx date?

# Step1, read from spreadsheet, clean up, and get statistics

In [265]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
from datetime import timedelta, date
import itertools

from matplotlib import pyplot
from pylab import *
import matplotlib.pyplot as plt

%matplotlib inline  
import seaborn as sns

In [266]:
#dx info
df1=pd.read_csv("./data/pts_ml_as_select_distinct_cc_person_id_cc_cancer_type_name_202102261747.csv", header=0)
#targeted table row pt list
df2=pd.read_excel("./data/pt_list_all_rows.xlsx", header=0) 
#psa info
df_psa=pd.read_csv("./data/PSA_values_labs_and_notes_temp_202103031711_fixed.csv", header=0) 
#fixed get rid of ">" or "<" symbols

df1.fillna('', inplace=True)
df1.replace(r'', np.NaN)
df1[:3]

df2.fillna('', inplace=True)
df2.replace(r'', np.NaN)
df2[:3]

df_psa.fillna('', inplace=True)
df_psa.replace(r'', np.NaN)
df_psa[df_psa['person_id']==339]

df1_pt['dxdate'] = pd.to_datetime(df1_pt['dxdate'])
df_psa['psa_date'] = pd.to_datetime(df_psa['psa_date'])

In [267]:
df1[:3]

,person_id,cancer_type_name,source_type,dxdate,dx_year,dx_month,date_of_birth,date_of_death,gender,race,ethnicity,ageatdx,psa,psa_date
0,2,PCA,ml,1/7/03,2003,1,10/15/33,4/22/17 0:00,Male,White,Not Hispanic or Latino,69,1.51,1/7/03
1,11,PCA,ml,3/19/12,2012,3,1/22/68,,Male,Black or African American,Not Hispanic or Latino,44,,
2,12,PCA,ml,10/24/12,2012,10,2/9/43,,Male,Black or African American,Not Hispanic or Latino,70,,


In [268]:
#need dx date info from this table 
df1_grouped=df1.groupby(["person_id"]).size().reset_index(name='counts')
list_pts_dx=df1["person_id"].to_list()

#need pt list for the result table 
list_pts_899row=df2["person_id"].to_list()
df2_grouped=df2.groupby(["person_id"]).size().reset_index(name='counts')
list_pts_399pts=df2_grouped["person_id"].to_list()

#need pt list that has PSA
df_psa_grouped=df_psa.groupby(["person_id"]).size().reset_index(name='counts')
list_pts_psa=df_psa_grouped["person_id"].to_list()

#find out overlapping lists for dx
intersection_set_dx=set.intersection(set(list_pts_399pts),set(list_pts_dx))
print("overlapping counts dx:")
print(len(intersection_set_dx))
overlap_list_dx=list(intersection_set_dx)

#find out overlapping lists for psa
intersection_set_psa=set.intersection(set(list_pts_399pts),set(list_pts_psa))
print("overlapping counts psa:")
print(len(intersection_set_psa))
overlap_list_psa=list(intersection_set_psa)

intersection_set_allthree=set.intersection(set(list_pts_399pts),set(list_pts_psa),set(list_pts_dx))
print("overlapping counts dx_psa_targeted_table:")
print(len(intersection_set_allthree))

overlapping counts dx:
376
overlapping counts psa:
367
overlapping counts dx_psa_targeted_table:
350


In [269]:
list_pts_399pts[:10]
list_pts_399pts[-10:]

[195757,
 196351,
 197112,
 198262,
 201667,
 202211,
 202841,
 203350,
 205050,
 205289]

In [270]:
index_list=df1.columns
index_list

Index(['person_id', 'cancer_type_name', 'source_type', 'dxdate', 'dx_year',
       'dx_month', 'date_of_birth', 'date_of_death', 'gender', 'race',
       'ethnicity', 'ageatdx', 'psa', 'psa_date'],
      dtype='object')

In [271]:
print(df1.shape)

(16018, 14)


In [272]:
#test out scenarioes
#[267, 339, 345, 435, 805]
#339 has no psa before dx

# The cell below is reserved for prototyping and spot checking 

In [273]:
y=8334
df1_pt=df1[(df1["person_id"] == y)]
df1_pt['dxdate'] = pd.to_datetime(df1_pt['dxdate'])

pt_dx_date=df1_pt["dxdate"].to_list()[0]
#df2 only provide the 899 row of pt list 
print(pt_dx_date)
df_psa_pt=df_psa[(df_psa["person_id"] == y)]
print(type(pt_dx_date))

2018-09-07 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [274]:
df_psa_pt[:5]

,person_id,psa_date,psa_value,matched_psa_date
1527,8334,2007-07-02,4.5,
1528,8334,2011-09-20,8.21,
1529,8334,2009-02-12,7.36,
1530,8334,2012-05-16,8.18,
1531,8334,2017-05-16,18.6,5/16/17


In [275]:
# for these 399 pts, how many had psa results before dx?

for y in list_pts_899row: 

    df1_pt=df1[(df1["person_id"] == y)]
    df1_pt['dxdate'] = pd.to_datetime(df1_pt['dxdate'])
    
    if df1_pt["dxdate"].empty:
        count=0

    if not df1_pt["dxdate"].empty:
        pt_dx_date=df1_pt["dxdate"].to_list()[0]
        df_psa_pt=df_psa[(df_psa["person_id"] == y)]
        df_psa_pt_before_dx=df_psa_pt[df_psa_pt.psa_date <= pt_dx_date]
        count=shape(df_psa_pt_before_dx)[0]
    #print(y,count)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [276]:
def closest(lst, K): 
      return lst[min(range(len(lst)), key = lambda i: abs(lst[i] - K))] 
    
#psa_date=closest(start_sorted_bin,pt_dx_date)

In [277]:
# for these 399 pts, how many had psa results before dx?
#sort all psa dates; find the last before dx; grab the dates, and all values on that date 
values=[]
result_list=[]

for y in list_pts_899row: 

    # pt must be in the dx_date list
    if y not in overlap_list_dx:
        count=0
        last_psa_date='NaN'
        values=['NaN']        

    if y in overlap_list_dx:
        df1_pt=df1[(df1["person_id"] == y)]
        df1_pt['dxdate'] = pd.to_datetime(df1_pt['dxdate'])
    
        if df1_pt["dxdate"].empty:
            count=0
            last_psa_date='NaN'
            values=['NaN']        

        if not df1_pt["dxdate"].empty:
            pt_dx_date=df1_pt["dxdate"].to_list()[0]
            df_psa_pt=df_psa[(df_psa["person_id"] == y)]
        
            df_psa_pt_before_dx=df_psa_pt[(df_psa_pt.psa_date <= pt_dx_date)&(df_psa_pt.psa_value.notnull())]
 
            each_pt_psa_date_before_dx = df_psa_pt_before_dx['psa_date'].to_list()
            sorted_each_pt_psa_before_dx=sorted(each_pt_psa_date_before_dx)
            count=len(sorted_each_pt_psa_before_dx)
        
            if len(sorted_each_pt_psa_before_dx)==0:
                last_psa_date='NaN'
                values=['NaN']
        
            if len(sorted_each_pt_psa_before_dx)>0:
                last_psa_date=closest(sorted_each_pt_psa_before_dx,pt_dx_date)
                #pick up all values for that date
                sub_set=df_psa_pt_before_dx[(df_psa_pt_before_dx['psa_date']==last_psa_date)]
                
                #sort and get rid of redundancies
                each_pt_info= sorted(sub_set.psa_value.to_list())
                each_pt_dates=list(dict.fromkeys(each_pt_info))

                values=each_pt_dates
                print(y,count,last_psa_date,values)
            
    result_per_pt=[y,pt_dx_date,count,last_psa_date,values]
    result_list.append(result_per_pt)

/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


3576 5 2016-08-22 00:00:00 ['0.98', '18.25', '5.37']
3576 5 2016-08-22 00:00:00 ['0.98', '18.25', '5.37']
3576 5 2016-08-22 00:00:00 ['0.98', '18.25', '5.37']
7532 10 2017-04-21 00:00:00 ['481', '481.04']
7867 2 2018-04-11 00:00:00 ['9.5']
7867 2 2018-04-11 00:00:00 ['9.5']
7867 2 2018-04-11 00:00:00 ['9.5']
7873 40 2015-09-15 00:00:00 ['13.85']
8260 2 2018-06-08 00:00:00 ['1.99', '21.9']
8334 23 2017-05-16 00:00:00 ['18.6']
8334 23 2017-05-16 00:00:00 ['18.6']
8334 23 2017-05-16 00:00:00 ['18.6']
8334 23 2017-05-16 00:00:00 ['18.6']
9071 1 2016-03-10 00:00:00 ['2.43']
9097 22 2015-07-22 00:00:00 ['1050.8']
9097 22 2015-07-22 00:00:00 ['1050.8']
9097 22 2015-07-22 00:00:00 ['1050.8']
11876 3 2016-07-18 00:00:00 ['0.94', '17.31', '5.43']
11876 3 2016-07-18 00:00:00 ['0.94', '17.31', '5.43']
11876 3 2016-07-18 00:00:00 ['0.94', '17.31', '5.43']
11876 3 2016-07-18 00:00:00 ['0.94', '17.31', '5.43']
11876 3 2016-07-18 00:00:00 ['0.94', '17.31', '5.43']
11876 3 2016-07-18 00:00:00 ['0.94', 

191902 4 2015-11-30 00:00:00 ['1.05', '18.75', '5.6']
191902 4 2015-11-30 00:00:00 ['1.05', '18.75', '5.6']
198262 2 2019-09-26 00:00:00 ['4.36']


In [278]:
df_psa_results=pd.DataFrame(result_list)
df_psa_results.columns=(['person_id', 'dx_date','Number_of_PSA_dates_with_PSA_values_before_dx','Last_PSA_date_before_dx', 'Last_PSA_values_before_dx'])

In [279]:
df_psa_results[200:203]

,person_id,dx_date,Number_of_PSA_dates_with_PSA_values_before_dx,Last_PSA_date_before_dx,Last_PSA_values_before_dx
200,17782,2015-07-16,4,2015-07-13 00:00:00,[1146.49]
201,18071,NaT,0,NaN,[NaN]
202,18071,NaT,0,NaN,[NaN]


In [280]:
len(df_psa_results)
len(list_pts_899row)

899

In [281]:
df_psa_results.to_csv("./data/pca_399pts_with_PSA_dates_and_values_before_dx.csv",encoding='utf-8',index=False)